In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import END, START
from langgraph.graph.state import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [5]:
os.environ["LANGSMITH_TRACING"]="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT="langchain_agents"

In [6]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("groq:llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F226834E50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F2267BED90>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from langchain_core.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b

@tool
def subtract_numbers(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

tools = [add_numbers, subtract_numbers]

tool_node = ToolNode(tools, name="add_numbers_tool")

In [9]:
llm_with_tools = llm.bind_tools(tools)

In [12]:
class State(TypedDict):
    """State for the graph."""
    messages: Annotated[list[BaseMessage], add_messages]

In [13]:
def call_llm_model(state:State):
    """Call the LLM model with the current state."""
    return {"messages":[llm_with_tools.invoke(state.messages)]}